# Searching for G4 motifs in protein coding genes 
Archishma Kavalipati 

started analysis 23/11/20

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
# this is the path to GENCODE annotations in lab cluster
%ls /proj/RNA_lab/GRCh38_gencode_v44/

basic.transcriptome.fa.gz
GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fa
GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fa.fai
gencode.v44.basic.annotation.gtf.gz
gencode.v44.transcriptome.fa.gz
genome.gtf
kallisto/
kallisto_index.sh
reformat_gencode_ensembl_annotation.sh
salmon/
salmon_index.sh
STAR/
star_index.sh
transcriptome_filter.py
transcripts_gtf.txt


In [3]:
# these are ENSEMBL annotations, which contain everything 
ensembl = "/proj/RNA_lab/Genomic_Indexes/Human/Homo_sapiens.GRCh38.108.chr.gtf"

# these are GENCODE basic annotations, parsed by Jose to include diff versions of same genes
# (closer to ENSEMBL format)
# I will be using these to find protein coding genes 
gencode = "/proj/RNA_lab/GRCh38_gencode_v44/genome.gtf"

### Following code didn't work out

In [4]:
## Not sure why I was trying to do this 

# gtf = None
# with open(gencode) as gtf:
#     gtf = gtf.readlines()[5:]

In [5]:
# gtf = pd.Series(gtf) # so we can use pd.apply() on the objects 

In [6]:
# gtf = gtf.apply(lambda x: x.split('\t'))

In [7]:
# # see output 
# print(gtf[0])

# # length 
# print(len(gtf))

In [8]:
# gtf = pd.DataFrame(row for row in gtf)

In [9]:
# gtf.columns = ['chr', 'annotations', 'type', 'start', 'end', 'score', 'strand', 'phase', 'info']

In [10]:
# gtf = gtf.drop(axis = 1, labels = ["score", "phase"])
# gtf

In [11]:
# info = gtf[["info"]] # get info out of key value pairs 

In [12]:
# def info_parser(info):
#     '''Takes in info string. Gets out argument values.'''
#     info = info.split(";")
#     vals = ["gene_id", "gene_version", "gene_type", "gene_name", "level", "tag"]
#     # sanity checking with if statements 
#     gene_id = info[0].split(vals[0]) if vals[0] in info[0] else ""
#     gene_version = info[1].split(vals[1]) if vals[1] in info[1] else ""
#     gene_type = info[2].split(vals[2]) if vals[2] in info[2] else ""
#     gene_name = info[3].split(vals[3]) if vals[3] in info[3] else ""
#     level = info[4].split(vals[4]) if vals[4] in info[4] else ""
#     tag = info[5].split(vals[5]) if vals[5] in info[5] else ""
    
#     print(gene_id[1])
#     parsed_vals = [x[1].strip(" ").strip("\"") for x in [gene_id, gene_version, gene_type, gene_name, level, tag]]
#     return(parsed_vals)

In [13]:
# type(info["info"])

In [14]:
# # try out the info parser on first line
# info_parser(info["info"][0])

In [15]:
# info["info"].apply(info_parser)

## Faster method
### I used the following code in the command line to only subset the protein coding genes from the gtf file
`awk '/protein_coding/ {print}' genome.gtf > ~/genome.proteinCoding.gtf`  
and then:  
`awk '{print $1, $2, $3, $4, $5, $7, $10}' genome.proteinCoding.gtf > genome.proteinCoding.subset.gtf`  
to get the output file:  
`genome.proteinCoding.subset.gtf`

anatomy of the file:  
| chr | annotation | gene_type | start | end | strand | gene_id |  

In [16]:
coding = pd.read_csv("/nas/longleaf/home/askav/rna/autoregulation/g4/genome.proteinCoding.subset.gtf", sep=" ", header=None)

In [17]:
coding[6] = coding[6].apply(lambda x: x.strip(';')) # get rid of trailing ';' character 

In [18]:
coding.columns = ['chr', 'annotation', 'type', 'start', 'end', 'strand', 'gene_id']

In [19]:
coding

,chr,annotation,type,start,end,strand,gene_id
0,chr1,HAVANA,gene,65419,71585,+,ENSG00000186092
1,chr1,HAVANA,transcript,65419,71585,+,ENSG00000186092
2,chr1,HAVANA,exon,65419,65433,+,ENSG00000186092
3,chr1,HAVANA,exon,65520,65573,+,ENSG00000186092
4,chr1,HAVANA,CDS,65565,65573,+,ENSG00000186092
...,...,...,...,...,...,...,...
1757648,chrM,ENSEMBL,gene,14747,15887,+,ENSG00000198727
1757649,chrM,ENSEMBL,transcript,14747,15887,+,ENSG00000198727
1757650,chrM,ENSEMBL,exon,14747,15887,+,ENSG00000198727
1757651,chrM,ENSEMBL,CDS,14747,15887,+,ENSG00000198727


In [20]:
coding['type'].value_counts()

exon              710377
CDS               654548
UTR               180420
stop_codon         64079
transcript         64057
start_codon        64019
gene               20046
Selenocysteine       107
Name: type, dtype: int64

### workflow
protein coding genes --> find coords --> get G4s using regex --> subset coords --> eCLIP BAMs corresponding to coords 

In [21]:
# don't use this one - this is all exonic sequences 
## there's also this source of "protein coding exons" in /proj/RNA_lab/Genomic_Indexes
# exons_bed = pd.read_csv("/proj/RNA_lab/Genomic_Indexes/Human/Protein_Coding_Exons.bed", sep="\t")
# exons_bed
# has more genes counted than my method 

In [25]:
pd.read_csv("/nas/longleaf/home/askav/rna/autoregulation/g4/seq_list_100.tsv", sep="\t", header=None)

,0,1,2,3,4,5
0,gene,1,923923,944575,+,GGGCGGCGGGGCGGGGGCTTGGGACCCCCG
1,gene,1,923923,944575,+,GGGAGGAGGGGCGCCGGGTGGGGACGCCCA
2,gene,1,923923,944575,+,GGGTGGGACGGGAAGCGGGCTGGGAAGTCGG
3,gene,1,923923,944575,+,GGGCAGGGTGGGTGGCAGGGAAGCTGT
4,gene,1,923923,944575,+,GGGGGCAGCCCAGGGTCAGGGCAGGGTCTCAGC
5,gene,1,923923,944575,+,GGGGTGGGGCAGGGGAGGGCTGAGCC
6,gene,1,923923,944575,+,GGGGGCCTGAGGGCGGGGTCGGGGCTGTGGGGCCAGAGG
7,gene,1,923923,944575,+,GGGCTCCGGGAAGGATGGGGTTCTCGGGAGGGAAG
8,gene,1,923923,944575,+,GGGCTTGTGGGGGGACGGGCTTGGGAGCCCCA
9,gene,1,923923,944575,+,GGGCCTTCTGGGGGCAGGGATGGGTATGACC
